In [8]:
import numpy as np
import time
from rr_python_sdk import cpp, MeasurementHead, msg_map_to_host

In [20]:
head = MeasurementHead("COM6")

Handshake acknowledged: <rr_python_sdk.mh_protocol_py.HandshakeToHostPayload object at 0x000002182B1237B0>


In [21]:
head.shakehands()

Handshake acknowledged: <rr_python_sdk.mh_protocol_py.HandshakeToHostPayload object at 0x000002182B14B570>


In [10]:
head.request_current_force_data()

In [ ]:
head.zero_forces()

In [ ]:
head.enter_load_cell_calibration_mode_normal()

In [ ]:
coeff_n = cpp.LoadCellCalibrationCoef()
coeff_n.OFFSET_B = 0b1111101000111111
coeff_n.GAIN_B = 0b1001001110110101

In [ ]:
head.write_calibration_coefficients_normal(coeff_n)

In [ ]:
head.enter_load_cell_calibration_mode_tangential()

In [ ]:
coeff_t = cpp.LoadCellCalibrationCoef()
coeff_t.OFFSET_B = 0b0
coeff_t.GAIN_B = 0b1001001110110101 # just half the gain, no offset

In [ ]:
head.write_calibration_coefficients_tangential(coeff_t)

In [7]:
head.request_angle()

In [22]:
head.request_acceleration_data()

In [14]:
head.request_orientation()

In [23]:
a = head.read()
match a.type:
    case cpp.MsgType_ToHost.ERR:
        print(f"Error {a.data.reason}")
    case cpp.MsgType_ToHost.FaultData:
        print(f"Fault {a.data.fault_status:0b}, {int(np.log2(a.data.fault_status))}")
    case cpp.MsgType_ToHost.ACK:
        print(f"Ack {a.data.msg_type}")
    case cpp.MsgType_ToHost.NoMessage:
        print("No message")
    case cpp.MsgType_ToHost.CurrentData:
        print(f"Force data: N {a.data.f_normal_mn}mN, T {a.data.f_tangential_mn}mN")
    case cpp.MsgType_ToHost.OrientationData:
        print(f"Orientation data: incline: {a.data.incline}, roll: {a.data.roll}")
    case _:
        print(a.data)

Error error_reason_t.E_ACCELEROMETER_NOT_ENOUGH_MEASUREMENTS


In [ ]:
print(f"Force data: N {a.data.f_normal_mn}mN, T {a.data.f_tangential_mn}mN")

In [ ]:
head.reset_to_defaults()

In [ ]:
head.exit_load_cell_calibration_mode_normal()

In [17]:
head.restart()

In [ ]:
f"{0b100100111011010:0X}"

In [7]:
while (True):
    head.ser.write(bytes([0xAA,0xAA,0x00,0x11,0x22]))
    time.sleep(0.1)

SerialException: WriteFile failed (PermissionError(13, 'Access is denied.', None, 5))

In [ ]:
status = 400
match status:
    case 400:
        print("Bad request")
    case 404:
        print ("Not found")

In [ ]:
head.request_acceleration_data()

In [ ]:
a = head.read()
print(a.data.x, a.data.y, a.data.z)

In [ ]:
head.restart()

In [ ]:
a.data.fault_status

In [ ]:
a = cpp.FaultStatusPayload()

In [ ]:
f"{0x1f:b}"

In [ ]:
0x6081

In [ ]:
a = cpp.HandshakeToDevicePayload()

In [ ]:
import rr_python_sdk as a

In [ ]:
a.msg_map_to_host

In [ ]:
head.read()

In [ ]:
payload = cpp.JogPayload()
payload.distance_mm = 1
head.jog_motor(payload)

In [ ]:
payload = cpp.AccelerationData()
payload.x = 1

In [ ]:
payload = cpp.HandshakeToDevicePayload()


In [ ]:
locals()['hs']

In [ ]:
my_head = head.MeasurementHead("COM5")

In [ ]:
my_head.ack()

In [ ]:
head.read()

In [ ]:
head.cpp.AccelerationData()

In [ ]:
head.cpp.MsgType_ToHost.

In [ ]:
# --- Force Data Point ---
force = cpp.ForceDataPoint()  # corresponds to force_data_point_t
force.device_status = 2
force.f_normal_mn = 123456
force.f_tangential_mn = -654321
force.millis_since_start = 500

# Serialize to bytes
# We'll use memoryview/bytearray approach
force_bytes = force.serialize()

# Prepend tag for ESP (0x00)
frame = bytes([0x00]) + force_bytes
ser.write(frame)
print("Sent force_data_point_t: 0x" + ' 0x'.join(f'{x:02X}' for x in frame)[0:])

In [ ]:
# AccelerationData
accel = cpp.AccelerationData()
accel.x = 1.1
accel.y = 2.2
accel.z = 3.3
accel.t = 4.4

accel_bytes = accel.serialize()
ser.write(bytes([0x01]) + accel_bytes)
print("Sent acceleration_data_t:", accel_bytes)

In [ ]:
payload = b"Hello ESP32!\r\n"

print(f"Sending {len(payload)} bytes: {payload}")
ser.write(payload)

# Read back the same number of bytes
rx = ser.read(len(payload))
print(f"Received {len(rx)} bytes: {rx}")

# Compare
if rx == payload:
    print("✅ Echo OK")
else:
    print("❌ Mismatch!")
    print("Sent :", binascii.hexlify(payload))
    print("Recv :", binascii.hexlify(rx))

In [ ]:
ser.close()

In [ ]:
head = sdk.MeasurmentHead(port = "COM11")
head.read_all_input()

# Test all functions
 (In order they are in the microcontroller code)

## Angle setting

In [ ]:
head.set_angle(10)

## Echo input

In [ ]:
head.echo(b"Hello")

## Read and write config data
Note, the default config will be reloaded when the device restarts

In [ ]:
config = head.read_config_data()
config.microsteps_per_step

In [ ]:
config.t_force_scale

In [ ]:
config.microsteps_per_step = 8

In [ ]:
head.update_config_data(config)

In [ ]:
config = head.read_config_data()

In [ ]:
config.microsteps_per_step

## Data streaming

In [ ]:
i = 0
points = []
head.read_all_input()
head.start_streaming_force_data(100)
while (i<200):
    points.append(head.read_force_data_point_from_stream())
    i+=1
head.stop_streaming_force_data()
#plt.plot([p.ms_since_start for p in points], [p.f_t_mn/p.f_n_mn for p in points], label="mu")
plt.plot([p.ms_since_start for p in points], [p.f_n_mn for p in points], label="Normal force (mn)")
plt.plot([p.ms_since_start for p in points], [p.f_t_mn for p in points], label="Tangential force (mn)")
plt.legend()

In [ ]:
head.stop_streaming_force_data()
head.read_all_input()

## Show and reset error data

In [ ]:
f"{head.read_error_status():016b}"

In [ ]:
head.clear_error_status()

## Setting target force

In [ ]:
head.set_target_force(100000)

## Home

In [ ]:
head.home()

## Jog

In [ ]:
head.jog(100)

## Toggle LEDS

In [ ]:
head.toggle_led(True)
time.sleep(0.5)
head.toggle_led(False)
time.sleep(0.5)
head.toggle_led(True)
time.sleep(0.5)
head.toggle_led(False)

## Change state to None

In [ ]:
head.stop_all()

## Send current data

In [ ]:
head.read_single_force_data_point()

## Restart

In [ ]:
head.restart()

In [ ]:
head.read_all_input()

## Start test

In [ ]:
head.maintain_force(100000)

In [ ]:
i = 0
points = []
head.read_all_input()
head.start_streaming_force_data(100)
while (i<200):
    points.append(head.read_force_data_point_from_stream())
    i+=1
head.stop_streaming_force_data()

centre=0
vals = np.sort([p.f_t_mn for p in points])
span = np.mean(vals[-50:])-np.mean(vals[:50])
plt.plot([p.ms_since_start for p in points], [p.f_n_mn for p in points], label="Normal force (mn)")
plt.plot([p.ms_since_start for p in points], [p.f_t_mn for p in points], label="Tangential force (mn)")
plt.plot([0,2000], [centre, centre], 'k:')
plt.plot([0,2000], [centre-span/2, centre-span/2], 'g-')
plt.plot([0,2000], [centre+span/2, centre+span/2], 'g-')
plt.legend()

In [ ]:
head.stop_all()

## Zero all forces

In [ ]:
head.home()

In [ ]:
head.zero_forces()

## Incline and roll

In [ ]:
head.read_all_input()
head.read_orientation_data()